In [1]:
import sys

print(sys.version)
from paddleocr import PaddleOCR, draw_ocr
import cv2

import tensorflow as tf
import numpy as np
import pandas as pd
import os







ocr = PaddleOCR(lang='en')

def find_closest_known_keys(unassigned_key, known_keys):
    # Sort known keys
    print(known_keys)
    known_keys = sorted(known_keys)
    # Initialize distances and closest keys
    closest_keys = []
    for k in known_keys:
         closest_keys.append((abs(int(k) - int(unassigned_key)), k))
    # Sort by distance
    closest_keys.sort()
    # Return the two closest known keys
    return [closest_keys[0][1], closest_keys[1][1]]

def label_text(text):
    # Define the two lists
    list1 = ['t', 'r', 'u', 'T', 'R', 'U']
    list2 = ['f', 'a', 'l', 's', 'F', 'A', 'L', 'S']
    
    # Count the matches for each list
    count1 = sum(text.count(char) for char in list1)
    count2 = sum(text.count(char) for char in list2)
    
    # Determine the label based on the counts
    if count1 > count2:
        return True
    else:
        return False



def assign_true_false_or_unknown(true_list, false_list, question_dict):
    # Initialize the final dictionary
    final_dict = {str(i): 'UNASSIGNED' for i in range(1, 11)}
    unassigned_keys=[]
    assigned_keys=[]
    
    # Iterate over each question and its y-coordinate
    for question, y in question_dict.items():
        # Compute absolute differences with true list
        true_differences= [y - ty for ty in true_list]
        true_abs_differences = [abs(y - ty) for ty in true_list]
        # Compute absolute differences with false list
        false_differences= [y - ty for ty in false_list]
        false_abs_differences = [abs(y - fy) for fy in false_list]


        # Find the minimum differences
  
        min_true_diff = min((diff for diff in true_differences if diff > 0), default=float('inf'))
        min_false_diff = min((diff for diff in false_differences if diff > 0), default=float('inf'))

        
        min_true_abs_diff=min(true_abs_differences) if true_differences else float('inf')
        min_false_abs_diff=min(false_abs_differences) if false_differences else float('inf')

        # Determine the smallest difference
        min_diff = min(min_true_diff, min_false_diff)
        min_abs_diff=min(min_true_abs_diff,min_false_abs_diff)


        # Assign the value based on the smallest difference
        if min_diff < 300:
            if min_true_diff < min_false_diff:
                final_dict[question] = True
                true_list.pop(true_differences.index(min_true_diff))
            else:
                final_dict[question] = False
                false_list.pop(false_differences.index(min_false_diff))
        else:
            # checking the abs diff option if nothing can find in positive diff option
            if min_abs_diff < 300:
                if min_true_abs_diff < min_false_abs_diff:
                    final_dict[question] = True
                    true_list.pop(true_abs_differences.index(min_true_abs_diff))
                else:
                    final_dict[question] = False
                    false_list.pop(false_abs_differences.index(min_false_abs_diff))
            
            else:
                final_dict[question] = 'NULL'
   
    print("the final dict without editing",final_dict)
    
    # solving the case when question number is not detected but answer is detected
    for key, value in final_dict.items():
        if value == "UNASSIGNED":
            unassigned_keys.append(key)
        else:
            assigned_keys.append(key)
    
    closest_keys_dict={}
    print("the unassigned keys are",unassigned_keys)
    print("the assigned keys are",assigned_keys)
    try:
        for key in unassigned_keys:
            print("the key is: ",key)
            if key=='1':
                question_dict[key]=question_dict['2']-300
            elif key=='10':
                question_dict[key]=question_dict['9']+300
            else:
                lesser_key, greater_key = find_closest_known_keys(key,assigned_keys)
                closest_keys_dict[key] = (lesser_key, greater_key)

                diff=int(greater_key)-int(lesser_key)
                print(lesser_key)
                print(greater_key)
                

                index=int(key)-int(lesser_key)-1
                lowest_adj_index=int((diff-1)/2)
                adj_mult=(-lowest_adj_index)+index
                
                if((diff-1)%2==0):
                    if(adj_mult>=0):
                        adj_mult=adj_mult+1
                
                print("the number which is used for adjusting ",adj_mult)
                question_dict[key]=((question_dict[lesser_key]+question_dict[greater_key])/2)+(300*adj_mult)

        

        for question, y in question_dict.items():
            if question in unassigned_keys:

                print("that question which is editted is",question)
                print("the value of y is",y)
                # Compute absolute differences with true list
                true_differences = [abs(y - ty) for ty in true_list]
                # Compute absolute differences with false list
                false_differences = [abs(y - fy) for fy in false_list]
                
                # Find the minimum differences
                min_true_diff = min(true_differences) if true_differences else float('inf')
                min_false_diff = min(false_differences) if false_differences else float('inf')
                
                # Determine the smallest difference
                min_diff = min(min_true_diff, min_false_diff)
                
                # Assign the value based on the smallest difference
                if min_diff < 350:
                    if min_true_diff < min_false_diff:
                        final_dict[question] = True
                        true_list.pop(true_differences.index(min_true_diff))
                    else:
                        final_dict[question] = False
                        false_list.pop(false_differences.index(min_false_diff))
                else:
                    final_dict[question] = 'NULL'
    
    
    except Exception as e:
        return final_dict

                           
    
    
    
    
    return final_dict





def process_and_save_image(image_path, output_folder , output_folder1):

    base_name = os.path.basename(image_path)
    image_cv = cv2.imread(image_path)
    image_height = image_cv.shape[0]
    image_width = image_cv.shape[1]
    # print(image_width)
    output = ocr.ocr(image_path)[0]

    # print(output)
    print("!------------------------------start-----------------------------------!")
    print("Started processing of the image :",base_name)
    boxes = [line[0] for line in output]
    texts = [line[1][0] for line in output]
    probabilities = [line[1][1] for line in output]

    image_boxes = image_cv.copy()

    for box,text in zip(boxes,texts):
      cv2.rectangle(image_boxes,(int(box[0][0]),int(box[0][1])),(int(box[2][0]),int(box[2][1])),(0,0,255),5)  #needs top left and bottom right to draw bounding box
      cv2.putText(image_boxes,text,(int(box[0][0]),int(box[0][1])),cv2.FONT_HERSHEY_SIMPLEX,4,(222,0,0),1)


    # saving the detection of all boxes
    output_file_name = f'detect_{base_name}'
    output_file_path = os.path.join(output_folder1, output_file_name)
    cv2.imwrite(output_file_path, image_boxes)


    for box, text in zip(boxes, texts):
      if text=='SN':
        num_l_x1=box[0][0]
        num_r_x1=box[2][0]+140
        print("left top x of SN:",num_l_x1)
        print("bottom right x of SN:",num_r_x1)
      elif text=='TRUE/FALSE':
        num_l_x2=box[0][0]-10
        num_r_x2=box[2][0]+20
        print("left top x of T/F:",num_l_x2)
        print("bottom right x of T/F:",num_r_x2)

    true_list=[]
    false_list=[]
    numbers_dict={}
    c=0
    prev=0

    cons_boxes_image=image_cv.copy()
    for box, text in zip(boxes, texts):

        box_top_left_x = int(box[0][0])
        
        box_bottom_right_x = int(box[2][0])
        box_bottom_right_y = int(box[2][1])
        box_width=box_bottom_right_x-box_top_left_x
        
        # Draw the adjusted bounding box
        if (num_l_x2 <=  box_bottom_right_x <= num_r_x2 and box_width<=1200  and 0 <= box_bottom_right_y <= image_height):
            print("entered in the t/f column ")
            
            if label_text(text)==True and text!='TRUE/FALSE':
                if(c==0):
                    print("first t/f detection")
                    print(text)
                    print(box)
                    prev=box[0][1]
                    true_list.append(int(box[0][1]))
                else:
                    
                    if(abs(box[0][1]-prev)>250):  #to avoid boxes in same row to overlap 
                        print(text)
                        print(box)
                        true_list.append(int(box[0][1]))
                        prev=box[0][1]
                c+=1
            
            elif label_text(text)==False and text!='TRUE/FALSE':
                if(c==0):
                    print("first t/f detection")
                    print(text)
                    print(box)
                    prev=box[0][1]
                    false_list.append(int(box[0][1]))
                else:
                    if(abs(box[0][1]-prev)>250):
                        print(text)
                        print(box)
                        false_list.append(int(box[0][1]))
                        prev=box[0][1]
                c+=1

            cv2.rectangle(cons_boxes_image,(int(box[0][0]),int(box[0][1])),(int(box[2][0]),int(box[2][1])),(0,0,255),5)  
            cv2.putText(cons_boxes_image,text,(int(box[0][0]),int(box[0][1])),cv2.FONT_HERSHEY_SIMPLEX,4,(222,0,0),1)
        
        elif(num_l_x1 <= box_bottom_right_x <= num_r_x1 and box_width<=1200 and 0 <= box_bottom_right_y <= image_height and text!='SN'):
                print("entered in the S/N column ")
                print(text)
                print(box)
                numbers_dict[text]=int(box[0][1])

                cv2.rectangle(cons_boxes_image,(int(box[0][0]),int(box[0][1])),(int(box[2][0]),int(box[2][1])),(0,0,255),5)  
                cv2.putText(cons_boxes_image,text,(int(box[0][0]),int(box[0][1])),cv2.FONT_HERSHEY_SIMPLEX,4,(222,0,0),1)
            
  
        


    # saving the specific region bounding boxes image
    output_file_name = f'final_tf_{base_name}'
    output_file_path = os.path.join(output_folder, output_file_name)
    cv2.imwrite(output_file_path, cons_boxes_image)

    
    
    print(numbers_dict)
    final_dict=assign_true_false_or_unknown(true_list,false_list,numbers_dict)

    
    print("------Printing the final dict ----------")
    print(final_dict)


   


    # df=pd.DataFrame(final_dict.items(),columns=['Q_No.','True/False'])

    
    # output_file_name = f'answers_{base_name}.csv'

    # output_file_path = os.path.join(output_folder, output_file_name)
    # df.to_csv(output_file_path,index=False)
    

    # print(f'DataFrame saved to {output_file_path}')
   


        
        
    


    predictions_file_path='pred_output.csv'
    actualAns_df=pd.read_csv('test_1/ModelAnswer.csv')

    marks=0

    for index, row in actualAns_df.iterrows():
        question_number = str(row["Question Number"])
        answer = row["Correct Answer"]
        if question_number in final_dict and final_dict[question_number]==answer:
            marks += 1

    print("Total Marks:", marks)
    image_name = base_name #Replace this with the actual image name
    marks_df = pd.DataFrame({"Filename": [image_name], "Marks": [marks]})

    # Append the marks DataFrame to the predictions file
    marks_df.to_csv(predictions_file_path, mode='a', header=False, index=False)






  




        






3.8.19 (default, Mar 20 2024, 19:58:24) 
[GCC 11.2.0]


2024-06-15 21:07:56.137991: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-06-15 21:07:58.231795: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


[2024/06/15 21:08:01] ppocr DEBUG: Namespace(alpha=1.0, alphacolor=(255, 255, 255), benchmark=False, beta=1.0, binarize=False, cls_batch_num=6, cls_image_shape='3, 48, 192', cls_model_dir='/home/adi_techbuddy/.paddleocr/whl/cls/ch_ppocr_mobile_v2.0_cls_infer', cls_thresh=0.9, cpu_threads=10, crop_res_save_dir='./output', det=True, det_algorithm='DB', det_box_type='quad', det_db_box_thresh=0.6, det_db_score_mode='fast', det_db_thresh=0.3, det_db_unclip_ratio=1.5, det_east_cover_thresh=0.1, det_east_nms_thresh=0.2, det_east_score_thresh=0.8, det_limit_side_len=960, det_limit_type='max', det_model_dir='/home/adi_techbuddy/.paddleocr/whl/det/en/en_PP-OCRv3_det_infer', det_pse_box_thresh=0.85, det_pse_min_area=16, det_pse_scale=1, det_pse_thresh=0, det_sast_nms_thresh=0.2, det_sast_score_thresh=0.5, draw_img_save_dir='./inference_results', drop_score=0.5, e2e_algorithm='PGNet', e2e_char_dict_path='./ppocr/utils/ic15_dict.txt', e2e_limit_side_len=768, e2e_limit_type='max', e2e_model_dir=None

In [2]:
# Define the path to the folder containing images
folder_path = 'test_1/Sample_Data'

# Ensure the output directory exists
output_folder = 'test_1/output'
output_folder1='test_1/detection'


for file_name in os.listdir(folder_path):
    # Construct the full file path
    file_path = os.path.join(folder_path, file_name)
    
    # Check if the file is an image (you can add more extensions if needed)
    if file_path.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp', '.tiff')):
        # Process and save the image
        process_and_save_image(file_path, output_folder,output_folder1)

print("Processing complete. Images saved to:", output_folder,output_folder1)

[2024/06/15 21:08:06] ppocr WARNING: Since the angle classifier is not initialized, it will not be used during the forward process
[2024/06/15 21:08:08] ppocr DEBUG: dt_boxes num : 40, elapsed : 0.7637438774108887
[2024/06/15 21:08:12] ppocr DEBUG: rec_res num  : 40, elapsed : 4.167423486709595
!------------------------------start-----------------------------------!
Started processing of the image : 20240328_150223(r).jpg
left top x of T/F: 5548.0
bottom right x of T/F: 6576.0
left top x of SN: 429.0
bottom right x of SN: 808.0
entered in the t/f column 
entered in the t/f column 
first t/f detection
Troue
[[5657.0, 545.0], [6119.0, 545.0], [6119.0, 748.0], [5657.0, 748.0]]
entered in the S/N column 
1
[[477.0, 643.0], [560.0, 592.0], [612.0, 674.0], [529.0, 725.0]]
entered in the t/f column 
forse
[[5682.0, 1073.0], [6102.0, 1073.0], [6102.0, 1252.0], [5682.0, 1252.0]]
entered in the S/N column 
2
[[495.0, 1179.0], [610.0, 1179.0], [610.0, 1309.0], [495.0, 1309.0]]
entered in the t/f 

<h2>Predicting accu

In [3]:
import pandas as pd

# pred_df=pd.read_csv('pred_output/pred_290.csv')
# pred_df=pd.read_csv('csvs/pred_output_350_reas.csv')
# pred_df=pd.read_csv('csvs/pred_output_350_reas.csv')
# actual_df=pd.read_csv('Marks.csv')
pred_df=pd.read_csv('test_1/csvs/test_reiterate.csv')
actual_df=pd.read_csv('test_1/Marks.csv')
correct=0
total=0

for index, row in pred_df.iterrows():
    filename=row["Filename"]
    actual_row=actual_df.loc[actual_df['Filename']==filename]
    actual_marks=actual_row['Total Marks'].values[0]
    pred_marks=row['Total Marks']
    correct+=pred_marks
    total+=actual_marks


print("the accuracy is: ",correct/total)

the accuracy is:  0.9134615384615384


<h1>Actual eval method

In [1]:
import pandas as pd

# pred_df=pd.read_csv('pred_output/pred_290.csv')
# pred_df=pd.read_csv('csvs/pred_output_350_reas.csv')
# actual_df=pd.read_csv('Marks.csv')

pred_df=pd.read_csv('csvs/pred_output_350_reas.csv')
actual_df=pd.read_csv('Marks.csv')
error=0

for index, row in pred_df.iterrows():
    filename=row["Filename"]
    actual_row=actual_df.loc[actual_df['Filename']==filename]
    actual_marks=actual_row['Total Marks'].values[0]
    pred_marks=row['Total Marks']
    error+=abs(pred_marks-actual_marks)

print(len(pred_df))
print(len(actual_df))

print("the mean absolute error is: ",error/len(pred_df))

28
28
the mean absolute error is:  0.5357142857142857


<h2>Sorting Rows 


In [3]:
import csv

pred_path='test_1/csvs/test_reiterate.csv'
actual_path='test_1/Marks.csv'

# Function to read CSV file into a list of dictionaries
def read_csv(filename):
    with open(filename, mode='r', newline='') as file:
        reader = csv.DictReader(file)
        data = list(reader)
    return data

# Function to write a list of dictionaries into a CSV file
def write_csv(filename, data, headers):
    with open(filename, mode='w', newline='') as file:
        writer = csv.DictWriter(file, fieldnames=headers)
        writer.writeheader()
        writer.writerows(data)

# Read the CSV files
csv1_data = read_csv(pred_path)
csv2_data = read_csv(actual_path)

# Assuming file2.csv specifies the order using a common 'ID' field
order_mapping = {row['Filename']: idx for idx, row in enumerate(csv2_data)}

# Sort csv1_data based on the order specified in csv2_data
csv1_data_sorted = sorted(csv1_data, key=lambda x: order_mapping.get(x['Filename']))

# Write the sorted data to a new CSV file
write_csv(pred_path, csv1_data_sorted, csv1_data[0].keys())
